In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300
import random
import csv
import pandas as pd
import h5py
from pysr import PySRRegressor  # Miles Cranmer's PySR
import torch  # PyTorch -- note that this should be loaded AFTER PySR
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor 
import matplotlib.cm as cm
# Own scripts:
%load_ext autoreload
%autoreload 2
import physics
import data
import nnc2p
# from nnc2p import NeuralNetwork # our own architecture
# Get dirs
import os
cwd = os.getcwd()# "Code" folder
master_dir = os.path.abspath(os.path.join(cwd, ".."))

Get the folder for the eos tables

In [2]:
eos_tables_folder = os.path.join(master_dir, "eos_tables")

# Introduction

We are going to explore symbolic regression as a way to gain insight into C2P aspects. We will use PySR, the Github repo can be found [here](https://github.com/MilesCranmer/PySR).

Read in an EOS table

In [3]:
# Our downloaded EOS tables here
first_table_filename      = "LS180_234r_136t_50y_analmu_20091212_SVNr26.h5"
second_table_filename     = "GShen_NL3EOS_rho280_temp180_ye52_version_1.1_20120817.h5"
third_table_filename      = "SLy4_0000_rho391_temp163_ye66.h5"
# Then specify which we are going to use here -- let's try the SLy4
eos_table_filename = third_table_filename

Read in the SLy4 EOS table using our py script:

In [4]:
eos_table = physics.read_eos_table(eos_table_filename)

In [5]:
# Small test to see the output of the EOS table
test_ye = eos_table["ye"][()][0]
test_temp = eos_table["logtemp"][()][0]
test_rho = eos_table["logrho"][()][0]
test_press = eos_table["logpress"][()][0, 0, 0]
print(f"For ye {test_ye}, log temp {test_temp}, log rho {test_rho}, we have log p: {test_press}.")

For ye 0.005, log temp -3.0, log rho 3.0239960056064277, we have log p: 17.99956975587081.


# PySR explorations

## Analytic Gamma law

### Import training data

In [6]:
df = physics.generate_data_as_df(200)
df

,rho,eps,v,p,D,S,tau
0,7.961337,0.236087,0.319649,1.253046,8.402148,3.949742,2.701290
1,5.979841,0.299793,0.409997,1.195142,6.556218,4.419662,3.028383
2,1.789420,1.948681,0.423389,2.324672,1.975190,3.921111,4.961391
3,9.306846,0.241474,0.293585,1.498240,9.735878,4.193450,3.049464
4,10.010703,0.463988,0.528883,3.096566,11.795413,13.034884,9.754071
...,...,...,...,...,...,...,...
195,7.702782,0.973514,0.611309,4.999177,9.733204,19.717146,17.521607
196,5.652622,0.787668,0.080313,2.968260,5.670941,1.056769,4.518943
197,2.695031,1.121314,0.292665,2.014650,2.818436,2.474755,3.622844
198,2.015569,0.735307,0.683622,0.988041,2.761673,5.756956,4.671540


### Can PySR learn the analytic $\Gamma$-law EOS?

In [7]:
# Input
X = np.dstack((df["rho"], df["eps"]))[0]
print(X.shape)
# Output
y = df["p"]
print(y.shape)

(200, 2)
(200,)


In [8]:
# Below is the standard set-up shown on PySR github repo
model = PySRRegressor(
    niterations=40,  # < Increase me for better results
    binary_operators=["+", "*"],
    unary_operators=[
        "cos",
        "exp",
        "sin",
        "inv(x) = 1/x",
        # ^ Custom operator (julia syntax)
    ],
    extra_sympy_mappings={"inv": lambda x: 1 / x},
    # ^ Define operator for SymPy as well
    loss="loss(prediction, target) = (prediction - target)^2",
    # ^ Custom loss function (julia syntax)
)

In [9]:
model.fit(X, y)

d:\Anaconda3\lib\site-packages\pysr\sr.py:1259: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
d:\Anaconda3\lib\site-packages\pysr\julia_helpers.py:151: UserWarning: `torch` was loaded before the Julia instance started. This may cause a segfault when running `PySRRegressor.fit`. To avoid this, please run `pysr.julia_helpers.init_julia()` *before* importing `torch`. For updates, see https://github.com/pytorch/pytorch/issues/78829
  warnings.warn(


PySRRegressor.equations_ = [
	   pick      score                                           equation  \
	0         0.000000                                                 x0   
	1         0.420195                                            exp(x1)   
	2         0.175832                                          (x0 * x1)   
	3         1.535634                                     (x0 * sin(x1))   
	4  >>>>  30.287430                            ((x0 * 0.6666667) * x1)   
	5         0.039924  (((x1 * 0.25803784) * (x0 * 2.7054994)) * 0.95...   
	
	           loss  complexity  
	0  9.241160e+00           1  
	1  6.070693e+00           2  
	2  5.091847e+00           3  
	3  1.096372e+00           4  
	4  7.696519e-14           5  
	5  6.560545e-14           9  
]

We can then print the analytic expression:

In [10]:
model.sympy()

0.6666667*x0*x1

### Can PySR learn the simple C2P?

In [17]:
df = physics.generate_data_as_df(1000)
df

,rho,eps,v,p,D,S,tau
0,1.528004,0.288118,0.459442,0.293497,1.720322,1.317179,0.853093
1,5.632820,1.100745,0.423803,4.133531,6.218926,8.248147,9.109778
2,4.455899,0.645595,0.539136,1.917803,5.290667,7.030875,5.832534
3,1.804941,0.006145,0.236600,0.007394,1.857686,0.457005,0.066474
4,5.432881,0.162764,0.113164,0.589519,5.468005,0.791726,0.938749
...,...,...,...,...,...,...,...
995,1.225224,1.221148,0.273365,0.997454,1.273741,1.098710,1.748013
996,0.886651,1.475485,0.005496,0.872160,0.886664,0.016857,1.308319
997,2.382949,1.593343,0.122966,2.531237,2.401172,1.087605,3.912371
998,9.317872,1.219796,0.292129,7.577268,9.742867,9.026161,13.577712


In [18]:
# Input
X = np.dstack((df["D"], df["S"], df["tau"]))[0]
print(X.shape)
# Output
y = df["p"]
print(y.shape)

(1000, 3)
(1000,)


In [31]:
model = PySRRegressor(
    niterations=200,
    # ^ number of iterations
    populations = 15,
    # ^ number of populations to run, default value 15
    population_size = 30,
    early_stop_condition=(
        "stop_if(loss, complexity) = loss < 1e-3 && complexity < 15"
        # ^ stop early if we find a "good" and "simple" equation
    ),
    # ^ population size, default 33
    binary_operators=["+", "*", "-", "^"],
    unary_operators=[
        "exp",
        "log",
        "sqrt",
        "neg",
        "inv(x) = 1/x"
        # ^ Custom operator (julia syntax)
    ],
    extra_sympy_mappings={"inv": lambda x: 1 / x},
    # ^ Define operator for SymPy as well
    loss="loss(prediction, target) = (prediction - target)^2",
    # ^ Custom loss function (julia syntax)
)

In [32]:
model.fit(X, y)

d:\Anaconda3\lib\site-packages\pysr\sr.py:1259: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


PySRRegressor.equations_ = [
	    pick     score                                           equation  \
	0         0.000000                                           3.490505   
	1         0.609810                                           sqrt(x2)   
	2         0.833614                                  (x2 ^ 0.68019384)   
	3         0.293849                            (x2 - (x1 * 0.6666658))   
	4         0.773863            ((x2 ^ 0.8975117) + (x1 * -0.40171713))   
	5         0.025583  ((x2 ^ 0.8989676) + ((x1 + 0.38503948) * -0.39...   
	6         0.282104  ((x2 - (x1 - (2.1141105 ^ log(x1)))) * 0.7067767)   
	7         0.069874  ((x2 - (x1 + (0.3394276 - (sqrt(x1) ^ 1.500181...   
	8         0.046445  ((x2 - ((x1 * 0.8953143) + (sqrt(x1 * x0) * -0...   
	9   >>>>  0.342677  ((x2 - ((x1 * 0.8808253) + (0.73212016 - sqrt(...   
	10        0.042481  ((x2 - ((x1 * 0.8737067) + (0.87068933 - sqrt(...   
	11        0.011066  ((x2 - ((x1 * 0.87068933) + (sqrt(0.87068933) ...   
	12      

In [27]:
model.equations_

,complexity,loss,score,equation,sympy_format,lambda_format
0,1,9.549363,0.000000,3.490504,3.49050400000000,PySRFunction(X=>3.49050400000000)
1,2,5.189640,0.609810,sqrt(x2),sqrt(x2),PySRFunction(X=>sqrt(x2))
2,3,2.254775,0.833614,(x2 ^ 0.68021053),x2**0.68021053,PySRFunction(X=>x2**0.68021053)
3,5,1.252765,0.293849,(x2 + (x1 * -0.66666174)),-0.66666174*x1 + x2,PySRFunction(X=>-0.66666174*x1 + x2)
4,6,1.121657,0.110545,abs(x2 + (x1 * -0.6738634)),Abs(0.6738634*x1 - x2),PySRFunction(X=>Abs(0.6738634*x1 - x2))
5,7,0.398951,1.033723,(((x2 * 1.7942535) - x1) * 0.42594182),-0.42594182*x1 + 0.76424760133137*x2,PySRFunction(X=>-0.42594182*x1 + 0.76424760133...
6,8,0.376517,0.057876,abs(((x2 * 1.7730448) - x1) * 0.43487522),Abs(0.43487522*x1 - 0.771053247469856*x2),PySRFunction(X=>Abs(0.43487522*x1 - 0.77105324...
7,9,0.321599,0.157660,((((x2 - -0.56282926) * 1.7466879) - x1) * 0.4...,-0.4176508*x1 + 0.72950559878532*x2 + 0.410587...,PySRFunction(X=>-0.4176508*x1 + 0.729505598785...
8,10,0.286597,0.115226,(((x2 * 1.7467062) - abs(x1 + -1.2373165)) * 0...,0.72950538161706*x2 - 0.4176463*Abs(x1 - 1.237...,PySRFunction(X=>0.72950538161706*x2 - 0.417646...
9,11,0.171904,0.511139,(((x1 ^ 0.66303796) + ((x1 - x2) * -0.7327512)...,x1**0.66303796 - 0.7327512*x1 + 0.7327512*x2 -...,PySRFunction(X=>x1**0.66303796 - 0.7327512*x1 ...


We can then print the analytic expression:

In [28]:
model.sympy()

x1**0.66303796 - 0.7327512*x1 + 0.7327512*x2 - 0.5902447

In [29]:
def mse_loss(y, pred):

    return np.mean((y-pred)**2)

Test if the model generalizes well?

In [30]:
test_df = physics.generate_data_as_df(10000)
test_X = np.dstack((test_df["D"], test_df["S"], test_df["tau"]))[0]
true_values = test_df["p"]
predictions = model.predict(test_X)
loss = mse_loss(true_values, predictions)
print(loss)

0.18144073842679312


# __TODO__ Tabulated EOS

In [37]:
# dat = physics.generate_tabular_data(eos_table, number_of_points = 100000, save_name = "SLy4_training_data")
# dat = physics.generate_tabular_data(eos_table, number_of_points = 20000, save_name = "SLy4_test_data")

Saving to  D:\Coding\master-thesis-AI\Data/SLy4_training_data.csv
Saving to  D:\Coding\master-thesis-AI\Data/SLy4_test_data.csv


Load data

In [41]:
df = pd.read_csv(os.path.join(master_dir, "Data/SLy4_training_data.csv"))
df

,rho,eps,v,temp,ye,p,D,S,tau
0,13.590663,19.381445,0.683568,-0.333333,0.045,31.985334,18.620245,580.748110,655.358209
1,6.457329,28.786040,0.041886,2.166667,0.545,34.766248,6.463001,13.269992,186.431092
2,11.323996,19.654200,0.558550,0.866667,0.535,30.418591,13.652070,310.329455,393.570454
3,8.723996,19.246855,0.009527,-0.300000,0.095,26.341137,8.724392,2.749496,167.935288
4,5.523996,29.186040,0.334705,2.033333,0.095,34.232914,5.862105,103.365923,195.482542
...,...,...,...,...,...,...,...,...,...
99995,3.323996,25.652376,0.577186,0.600000,0.435,28.498601,4.070468,125.881369,157.178893
99996,9.757329,19.124371,0.376611,-2.900000,0.295,27.371010,10.532847,140.768515,238.842241
99997,9.590663,19.114310,0.266570,-0.833333,0.305,27.176186,9.950723,90.427775,207.064143
99998,14.690663,20.976110,0.476948,2.233333,0.485,35.391058,16.714229,326.155098,461.688397


For comparison, this was the table we trained on for the ideal gas EOS:

In [42]:
old_df = pd.read_csv(os.path.join(master_dir, "Data/ideal_gas_c2p_training_data.csv"))
old_df

,rho,eps,v,p,D,S,tau
0,0.662984,0.084146,0.218802,0.037192,0.679448,0.173724,0.077335
1,8.565808,0.205945,0.657351,1.176059,11.366755,13.318537,7.718100
2,4.387112,1.598809,0.021593,4.676103,4.388135,0.347321,7.020631
3,5.337054,0.530803,0.351307,1.888615,5.700396,4.031171,3.885760
4,1.133895,0.786717,0.079475,0.594703,1.137493,0.209600,0.905115
...,...,...,...,...,...,...,...
79995,8.101834,0.428605,0.616897,2.314990,10.294002,13.832316,9.813427
79996,7.841014,1.125480,0.209087,5.883268,8.018242,4.930289,9.678536
79997,4.628822,0.194190,0.237759,0.599248,4.765476,1.544018,1.129323
79998,9.913117,1.152242,0.477216,7.614874,11.280468,17.889657,18.592193


# Define and train architecture

## Define architecture and prepare data

In [76]:
class NNC2P(nn.Module):
    """
    Implements a two-layered neural network for the C2P conversion, using tabulated SLy4 EOS.
    """
    def __init__(self, h1: int = 600, h2: int = 200, reg: bool = False) -> None:
        """
        Initialize the neural network class.
        :param h1: Size (number of neurons) of the first hidden layer.
        :param h2: Size (number of neurons) of the second hidden layer.
        """
        # Call the super constructor first
        super(NNC2P, self).__init__()
        
        # For convenience, save the sizes of the hidden layers as fields as well
        self.h1 = h1
        self.h2 = h2
        
        # Initialize empty mean and std:
        self.mean = torch.Tensor([0, 0, 0])
        self.std = torch.Tensor([1, 1, 1])

        # Add field to specify whether or not we do regularization
        self.regularization = reg

        # Define the layers:
        self.linear1 = nn.Linear(3, h1)
        self.linear2 = nn.Linear(h1, h2)
        self.linear3 = nn.Linear(h2, 1)
        
    def normalize(x):
        
        return (x-self.mean)/self.std

    def forward(self, x):
        """
        Computes a forward step given the input x.
        :param x: Input for the neural network.
        :return: x: Output neural network
        """
        x = self.normalize(x)

        x = self.linear1(x)
        x = torch.sigmoid(x)
        x = self.linear2(x)
        x = torch.sigmoid(x)
        x = self.linear3(x)
        return x

Get the training data as DataSet and DataLoader objects:

In [65]:
# Read data as pandas dataframes
train_df = pd.read_csv(os.path.join(master_dir, "Data/SLy4_training_data.csv"))
test_df = pd.read_csv(os.path.join(master_dir, "Data/SLy4_test_data.csv"))
# Convert to PyTorch Datasets as we defined them
train_dataset = data.CustomDataset(train_df)
test_dataset = data.CustomDataset(test_df)
# Then create dataloaders, with batch size 32, from datasets
train_dataloader = DataLoader(train_dataset, batch_size=32)
test_dataloader = DataLoader(test_dataset, batch_size=32)

In [74]:
train_dataloader.dataset.mean

tensor([ 10.6174, 171.6808, 283.8405])

In [67]:
test = train_dataset.features
test[0]

tensor([ 18.6202, 580.7481, 655.3582])

In [68]:
train_dataset.__getitem__(0)

(tensor([1.8245, 2.6370, 2.5105]), tensor([31.9853]))

In [71]:
print(train_dataset.mean)
print(train_dataset.std)

tensor([ 10.6174, 171.6808, 283.8405])
tensor([  4.3864, 155.1245, 147.9844])


Create a new instance of the NNC2P:

In [45]:
model = NNC2P()

Create a trainer object from it:

In [79]:
# Set the learning rate to 1e-3
trainer = nnc2p.Trainer(model, 0.1, train_dataloader=train_dataloader, test_dataloader=test_dataloader)

## Train the network

In [80]:
trainer.train()

Training the model for 500 epochs.

 Epoch 0 
 --------------
Train loss: 2.84E+01
Test  loss: 2.86E+01

 Epoch 1 
 --------------
Train loss: 2.92E+01
Test  loss: 2.94E+01

 Epoch 2 
 --------------
Train loss: 2.94E+01
Test  loss: 2.96E+01

 Epoch 3 
 --------------
Train loss: 2.93E+01
Test  loss: 2.95E+01

 Epoch 4 
 --------------
Train loss: 2.93E+01
Test  loss: 2.95E+01

 Epoch 5 
 --------------
Train loss: 2.93E+01
Test  loss: 2.95E+01

 Epoch 6 
 --------------
Train loss: 2.93E+01
Test  loss: 2.95E+01

 Epoch 7 
 --------------
Train loss: 2.93E+01
Test  loss: 2.95E+01

 Epoch 8 
 --------------
Train loss: 2.93E+01
Test  loss: 2.95E+01

 Epoch 9 
 --------------
Train loss: 2.93E+01
Test  loss: 2.95E+01

 Epoch 10 
 --------------
Adapting learning rate to 0.05
Train loss: 2.93E+01
Test  loss: 2.95E+01

 Epoch 11 
 --------------
Train loss: 2.26E+01
Test  loss: 2.26E+01

 Epoch 12 
 --------------
Train loss: 2.26E+01
Test  loss: 2.26E+01

 Epoch 13 
 --------------
Train 

KeyboardInterrupt: 

__TODO__ do hyperparameter search over learning rate?

__TODO__ is the normalization done correctly?

# Archive

In [70]:
# Open the HDF5 file
with h5py.File(eos_table_filename, 'r') as file:
    eos_table = file
    Abar = file["Abar"][()]
    Albar = file["Albar"][()]
    Xa = file["Xa"][()]
    Xh = file["Xh"][()]
    Xn = file["Xn"][()]
    Xp = file["Xp"][()]
    Zbar = file["Zbar"][()]
    cs2 = file["cs2"][()]
    dedt = file["dedt"][()]
    dpderho = file["dpderho"][()]
    dpdrhoe = file["dpdrhoe"][()]
    energy_shift = file["energy_shift"][()]
    entropy = file["entropy"][()]
    gamma = file["gamma"][()]
    logenergy = file["logenergy"][()]
    logpress = file["logpress"][()]
    logrho = file["logrho"][()]
    logtemp = file["logtemp"][()]
    mu_e = file["mu_e"][()]
    mu_n = file["mu_n"][()]
    muhat = file["muhat"][()]
    munu = file["munu"][()]
    pointsrho = file["pointsrho"][()]
    pointstemp = file["pointstemp"][()]
    pointsye = file["pointsye"][()]
    u = file["u"][()] ## these don't exist???
    r = file["r"][()]
    ye = file["ye"][()]
# Print message
print(f"We successfully loaded the EOS table. We have {pointsrho[0]} rho, {pointstemp[0]} temp, {pointsye[0]} ye points.")
print(f"We have {pointsrho[0]*pointstemp[0]*pointsye[0]} data points.")

We successfully loaded the EOS table. We have 391 rho, 163 temp, 66 ye points.
We have 4206378 data points.
